In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TestModule(nn.Module):
    def __init__(self, in_dim, h_dim, out_dim):
        super().__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim

        self.fc1 = nn.Linear(in_dim, h_dim, bias=False)
        self.fc2 = nn.Linear(h_dim, out_dim, bias=False)

    def forward(self, x):
        x = self.fc1(x)
        x = F.silu(x)
        x = torch.triu(x)
        x = self.fc2(x)
        x = torch.sin(x)
        return x


in_dim, h_dim, out_dim = 4092, 2048, 256
model = TestModule(in_dim, h_dim, out_dim)
test_input = torch.rand((16, in_dim))

In [2]:
for _ in range(1000):
    _ = model(test_input)

In [3]:
from proteus import proteus, proteus_no_compile
model = proteus(model)
_ = model(test_input)

TypeError: functional_call() missing 1 required positional argument: 'named_buffers'

In [ ]:
from diffusers import StableDiffusion3Pipeline

pipe = StableDiffusion3Pipeline.from_pretrained(
        "stabilityai/stable-diffusion-3.5-medium", torch_dtype=torch.float16
    )
    pipe = pipe.to("mps")
    # pipe.transformer = proteus_no_compile(pipe.transformer)

    image = pipe(
        "A capybara holding a sign that reads Hello World",
        num_inference_steps=5,
        guidance_scale=4.5,
    ).images[0]
    image.save("capybara.png")
